# Call dialogflow with python api
## precondition
you have to download your json key, and ***export GOOGLE_APPLICATION_CREDENTIALS="/mnt/d/code/sabala/weather-e6aad-0371e7c946bc.json"***


In [20]:
# check firewall
!rm index.html*
!wget www.google.com

--2022-05-08 16:49:25--  http://www.google.com/
Connecting to 172.26.64.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [ <=>                ]  15.91K  --.-KB/s    in 0.03s   

2022-05-08 16:49:25 (605 KB/s) - ‘index.html’ saved [16288]



In [21]:
import uuid
from google.cloud import dialogflow

# session format: 'projects/*/locations/*/agent/environments/*/users/*/sessions/*'.
def get_session(project_id, session_id, env=None):
    """
    Using the same `session_id` between requests allows continuation
    of the conversation.
    :return: session is a str
    """
    if env is None:
        session_client = dialogflow.SessionsClient()
        session = session_client.session_path(project_id, session_id)
        return session
    else:
        assert isinstance(env, str)
        return f"projects/{project_id}/agent/environments/{env}/sessions/{session_id}"
    

# [START dialogflow_detect_intent_text]
def detect_intent_texts(project_id, session_id, text, language_code, env=None):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    session= get_session(project_id, session_id, env=env)
    print("Session path: {}\n".format(session))

    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)

    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )   

    return response


In [22]:
# set credentials, this is a must

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="/mnt/d/code/sabala/mega-sabala-9ibe-940e7527ac9b.json"

## examples of making session url

In [24]:
from google.cloud import dialogflow

project_id = "sabala-348110"

session_id = str(uuid.uuid4())
# texts = ["Me toque uma música", "toca nos 80", "Tocar música clássica", "parar música"][-1:]
language_code = "en-US"

session = get_session(project_id, session_id, env='new')
session


'projects/sabala-348110/agent/environments/new/sessions/4ae77438-4c3e-4685-ac7a-5c29a6ae2111'

# visit Mega Agent and print

In [17]:
project_id = "mega-sabala-9ibe"

session_id = str(uuid.uuid4())
texts = ["how's the weather today", "Você pode me dizer a maneira mais fácil de ganhar dinheiro?"]
texts = ["parar música"]
language_code = "pt-BR"
response = detect_intent_texts(
        project_id, session_id, texts, language_code
)

Session path: projects/mega-sabala-9ibe/agent/sessions/c2b07a76-4533-4181-b6a3-2e16b034f294



In [20]:
response.output_audio = "None"
print("=" * 20)
print("Query text: {}".format(response.query_result.query_text))
print(
    "Detected intent: {} (confidence: {})\n".format(
        response.query_result.intent.display_name,
        response.query_result.intent_detection_confidence,
    )
)
print("Fulfillment text: {}\n".format(response.query_result.fulfillment_text))
print("Fulfillment Full: \n{}\n".format(response))
print(type(response))

Query text: how's the weather today
Detected intent: Currency_ConversionA (confidence: 0.6612323522567749)

Fulfillment text: Desculpe, não posso atualizar as taxas de câmbio no momento.

Fulfillment Full: 
response_id: "bd75e32e-6e5c-4818-9df6-61dbadd65faa-74fe87bc"
query_result {
  query_text: "how\'s the weather today"
  parameters {
    fields {
      key: "amount"
      value {
        string_value: ""
      }
    }
    fields {
      key: "currency1"
      value {
        string_value: "D\303\263lar de Singapura"
      }
    }
  }
  all_required_params_present: true
  fulfillment_text: "Desculpe, n\303\243o posso atualizar as taxas de c\303\242mbio no momento."
  fulfillment_messages {
    text {
      text: "Desculpe, n\303\243o posso atualizar as taxas de c\303\242mbio no momento."
    }
  }
  webhook_payload {
    fields {
      key: "google"
      value {
        struct_value {
          fields {
            key: "expectUserResponse"
            value {
              bool_val

# visit media/Music Agent

## visit the default one: draft

In [25]:
project_id = "sabala-348110"

session_id = str(uuid.uuid4())
texts = ["Me toque uma música", "toca nos 80", "Tocar música clássica", "parar música"]
language_code = "en-US"
response = detect_intent_texts(
        project_id, session_id, texts[-1], language_code
)

Session path: projects/sabala-348110/agent/sessions/2a3537a5-0a0f-4892-95a8-1758396c0862



In [26]:
print( [(_, type(_)) for _ in dir(response) if not _.startswith("_")] )
response.output_audio = "None"
print(response.query_result.intent)

[('output_audio', <class 'str'>), ('output_audio_config', <class 'str'>), ('query_result', <class 'str'>), ('response_id', <class 'str'>), ('webhook_status', <class 'str'>)]
name: "projects/sabala-348110/agent/intents/4bde314e-8feb-4ad1-b1fc-3fe451a7b431"
display_name: "PlayMusic"
end_interaction: true



## visit active version in a specified environment

In [27]:
response = detect_intent_texts(
        project_id, session_id, texts[-1], language_code, env='new'
)

Session path: projects/sabala-348110/agent/environments/new/sessions/2a3537a5-0a0f-4892-95a8-1758396c0862



# Webhook handle
* https://github.com/googleapis/python-dialogflow/blob/HEAD/samples/snippets/webhook.py

In [ ]:
# TODO: change the default Entry Point text to handleWebhook

def handleWebhook(request):

    req = request.get_json()

    responseText = ""
    intent = req["queryResult"]["intent"]["displayName"]

    if intent == "Default Welcome Intent":
        responseText = "Hello from a GCF Webhook"
    elif intent == "get-agent-name":
        responseText = "My name is Flowhook"
    else:
        responseText = f"There are no fulfillment responses defined for Intent {intent}"

    # You can also use the google.cloud.dialogflowcx_v3.types.WebhookRequest protos instead of manually writing the json object
    res = {"fulfillmentMessages": [{"text": {"text": [responseText]}}]}

    return res

In [ ]:
handleWebhook(response)